In [177]:
import sympy as sp
from IPython.display import display, Math
import time
start_time = time.time()

In [178]:
# x, y sú súradnice bodov v R^2
x, y = sp.symbols('x y')
# t je parameter z intervalu I
t = sp.Symbol('t')
# a, b sú konštanty pre škálovanie elipsoidu, a - dotykový smer, b - normálový smer
a, b = sp.symbols('a b')
# všetky ostatné symboly sú parametre v parametrizácii krivky parametric_curve
c, d = sp.symbols('c d')

In [179]:
# Vstupné parametre
# interval vyčíslenia krivky
t_min = -10
t_max = 10
# parametre škálovania
a_0 = 2
b_0 = 1
# parametre parametrizácie
c_0 = 2
d_0 = 1

# parametrizácia krivky
parametric_curve = sp.Matrix([sp.cos(t),sp.sin(t)]) 
curve_name = 'circle'

In [180]:
# Vypočíta dotyčnicu a normálu pre parametric_curve
def local_coordinate_system(parametric_curve): 
    t = sp.Symbol('t') 
    parametric_curve_first_derivation = sp.diff(parametric_curve, t)
    T = parametric_curve_first_derivation / sp.sqrt(parametric_curve_first_derivation.dot(parametric_curve_first_derivation))
    T = sp.simplify(T)
    N = sp.Matrix([-T[1], T[0]])
    N = sp.simplify(N)
    return T, N

# Vyrobí inverznú maticu prechodu 
def transition_matrix(T, N):
    P = sp.Matrix.hstack(T, N)
    return P.T

def matrix_of_quadrics(A, B, C, D, E, F):
    row1 = [A, B/2, D/2]
    row2 = [B/2, C, E/2]
    row3 = [D/2, E/2, F]
    matrix = sp.Matrix([row1, row2, row3])
    return matrix

# Krivost krivky
def curvature(parametric_curve):
    # Compute derivatives
    parametric_curve_prime = sp.diff(parametric_curve, t)
    parametric_curve_double_prime = sp.diff(parametric_curve_prime, t)

    # Compute curvature
    numerator = sp.Abs(parametric_curve_prime[0] * parametric_curve_double_prime[1] - parametric_curve_double_prime[0] * parametric_curve_prime[1])
    denominator = (parametric_curve_prime[0]**2 + parametric_curve_prime[1]**2)**(3/2)
    return numerator / denominator

In [181]:
# Výpočet vektorov
T, N = local_coordinate_system(parametric_curve)
# Zostavenie matice
P = transition_matrix(T, N)
# Maticou P vynásobíme vektor (X-parametric_curve) a tak zmeníme bázu elipsy
vector = sp.Matrix([x - parametric_curve[0], y - parametric_curve[1]])
result_vector = P * vector
ellipse_in_new_basis = result_vector[0]**2/a**2 + result_vector[1]**2/b**2 - 1
# Derivácia jednoparametrického systému elíps 
derivation = sp.diff(ellipse_in_new_basis, t)

In [182]:
# Výpočet, o aký typ kužeľosečky ide
# Najprv potrebujeme extrahovať koeficienty z derivácie
expanded_expression = sp.expand(derivation)
A = expanded_expression.coeff(x**2)
B = expanded_expression.coeff(x*y)
C = expanded_expression.coeff(y**2)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x*y, x*z,..
coef_x = expanded_expression.coeff(x)
terms_x = sp.Add(*coef_x.args) if isinstance(coef_x, sp.Add) else coef_x
terms_x_filtered = [term for term in sp.Add.make_args(terms_x) if not term.has(y)]
D = sp.Add(*terms_x_filtered)

coef_y = expanded_expression.coeff(y)
terms_y = sp.Add(*coef_y.args) if isinstance(coef_y, sp.Add) else coef_y
terms_y_filtered = [term for term in sp.Add.make_args(terms_y) if not term.has(x)]
E = sp.Add(*terms_y_filtered)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x, y, z
terms = sp.Add(*expanded_expression.args) if isinstance(expanded_expression, sp.Add) else expanded_expression
terms_filtered = [term for term in sp.Add.make_args(terms) if not term.has(x) and not term.has(y)]
F = sp.Add(*terms_filtered)

matrix_of_derivation = matrix_of_quadrics(A, B, C, D, E, F)

# Print koeficientov
print("Coefficient of x^2:")
display(Math(sp.latex(sp.simplify(A))))
print(sp.latex(sp.simplify(A)))
print("Coefficient of y^2:")
display(Math(sp.latex(sp.simplify(C))))
print(sp.latex(sp.simplify(C)))
print("Coefficient of xy:") 
display(Math(sp.latex(sp.simplify(B))))
print(sp.latex(sp.simplify(B)))
print("Coefficient of x:")
display(Math(sp.latex(sp.simplify(D))))
print(sp.latex(sp.simplify(D)))
print("Coefficient of y:")
display(Math(sp.latex(sp.simplify(E))))
print(sp.latex(sp.simplify(E)))
print("Coefficient of constant term:")
display(Math(sp.latex(sp.simplify(F))))
print(sp.latex(sp.simplify(F)))

# Typ kužeľosečky
determinant = sp.det(matrix_of_derivation)
subdeterminant = A*C-B**2/4

Coefficient of x^2:


<IPython.core.display.Math object>

- \frac{\sin{\left(2 t \right)}}{b^{2}} + \frac{\sin{\left(2 t \right)}}{a^{2}}
Coefficient of y^2:


<IPython.core.display.Math object>

\frac{\sin{\left(2 t \right)}}{b^{2}} - \frac{\sin{\left(2 t \right)}}{a^{2}}
Coefficient of xy:


...

\frac{2 \sin{\left(t \right)}}{b^{2}}
Coefficient of y:


<IPython.core.display.Math object>

- \frac{2 \cos{\left(t \right)}}{b^{2}}
Coefficient of constant term:


<IPython.core.display.Math object>

0


In [183]:
# Vektory krivky a dalsie funkcie
parametric_curve_prime = sp.simplify(sp.diff(parametric_curve, t))
norm = sp.simplify(sp.sqrt(parametric_curve_prime[0]**2+parametric_curve_prime[1]**2))
#print('Norma: ' + str(sp.latex(norm)))

parametric_curve_double_prime = sp.diff(parametric_curve_prime, t)
alpha = parametric_curve_prime[1]*(parametric_curve_double_prime[0]*parametric_curve_prime[1]-parametric_curve_prime[0]*parametric_curve_double_prime[1])
beta = parametric_curve_prime[0]*(parametric_curve_double_prime[1]*parametric_curve_prime[0]-parametric_curve_double_prime[0]*parametric_curve_prime[1])
gamma = -alpha*parametric_curve[0]-beta*parametric_curve[1]-(b**2*(parametric_curve_prime[0]**2+parametric_curve_prime[1]**2)**2)/(b**2-a**2)

# Funkcia vzdialenosti
rho = abs(alpha*parametric_curve[0]+beta*parametric_curve[1]+gamma)/(sp.sqrt(alpha**2+beta**2))

# Pomer skalovania
ratio = abs(b/a**2 - b**2)

In [184]:
# Rozklad derivacie na priamky
p_t = parametric_curve_prime[0]*x+parametric_curve_prime[1]*y-(parametric_curve_prime[0]*parametric_curve[0]+parametric_curve_prime[1]*parametric_curve[1])
q_t = alpha*x+beta*y+gamma

curvature = sp.simplify(curvature(parametric_curve))

In [185]:
# Ukončenie merania času
end_time = time.time()
elapsed_time = end_time - start_time

In [186]:
# Zápis do súboru
print(parametric_curve)
print("Systém: ")
system = sp.latex(sp.simplify(ellipse_in_new_basis))
display(Math(system))

print("Derivácia sytému: ")
der = sp.latex(sp.simplify(derivation))
display(Math(der))

print("Determinant derivácie: ")
display(Math(sp.latex(sp.simplify(determinant))))

expression = sp.latex(sp.simplify(subdeterminant))
display(Math(expression))

print('Krivosť: ' + sp.latex(curvature))
print('Pomer: ' + sp.latex(ratio))
print('Rho: ' + sp.latex(rho))
print('p_t: ' + sp.latex(sp.simplify(p_t)))
print('q_t: ' + sp.latex(sp.simplify(q_t)))

print('Dotykový vektor: (' + str(sp.latex(parametric_curve_prime[0]/norm)) + ', ' + str(sp.latex(parametric_curve_prime[1]/norm))+ ')')
print('Normálový vektor: (' + str(sp.latex(parametric_curve_prime[1]/norm)) + ', ' + str(sp.latex(parametric_curve_prime[0]/norm)) + ')')

print("Čas: ")
print(elapsed_time, "seconds")


Matrix([[cos(t)], [sin(t)]])
Systém: 


<IPython.core.display.Math object>

Derivácia sytému: 


<IPython.core.display.Math object>

Determinant derivácie: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Krivosť: 1
Pomer: \left|{b^{2} - \frac{b}{a^{2}}}\right|
Rho: \frac{\left|{\frac{b^{2} \left(\sin^{2}{\left(t \right)} + \cos^{2}{\left(t \right)}\right)^{2}}{a^{2} - b^{2}}}\right|}{\sqrt{\left(- \sin^{2}{\left(t \right)} - \cos^{2}{\left(t \right)}\right)^{2} \cos^{2}{\left(t \right)} + \left(\sin^{2}{\left(t \right)} + \cos^{2}{\left(t \right)}\right)^{2} \sin^{2}{\left(t \right)}}}
p_t: - x \sin{\left(t \right)} + y \cos{\left(t \right)}
q_t: \frac{b^{2} + \left(a^{2} - b^{2}\right) \left(- x \cos{\left(t \right)} - y \sin{\left(t \right)} + 1\right)}{a^{2} - b^{2}}
Dotykový vektor: (- \sin{\left(t \right)}, \cos{\left(t \right)})
Normálový vektor: (\cos{\left(t \right)}, - \sin{\left(t \right)})
Čas: 
1.5121018886566162 seconds


In [187]:
with open('2D_examples/' + curve_name + '.txt', "w") as file:
    # Write content to the file
    file.write(f"{curve_name} {parametric_curve}\n")
    file.write('System:\n')
    file.write(f"{system} = 0\n")
    file.write('Derivation:\n')
    file.write(f"{der} = 0\n")
    file.write('Time:\n')
    file.write(f"{elapsed_time}\n")
    file.write('Determinant:\n')
    file.write(f"{determinant}\n")
    file.write('Subdeterminant:\n')
    file.write(f"{expression}\n")
    file.write('Curvature:\n')
    file.write(f"{sp.latex(curvature)}\n")
    file.write('Ratio:\n')
    file.write(f"{sp.latex(ratio)}\n")
    file.write('Rho:\n')
    file.write(f"{sp.latex(rho)}\n")
    file.write('p_t:\n')
    file.write(f"{sp.latex(p_t)}\n")
    file.write('q_t:\n')
    file.write(f"{sp.latex(q_t)}\n")